In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import statsmodels.tsa.filters.hp_filter as hpfilter
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [2]:
start_year = 1970
end_year = 2023
year_obs = end_year - start_year +1

In [3]:
dfWEOraw = pd.read_excel('WEOApr2024all.xlsx',sheet_name='WEOApr2024all')

country_code = set(dfWEOraw['WEO Country Code'])
num_country = len(country_code)

year_list = list(range(start_year, end_year+1))*num_country

dictWEO = {}

country_list = []

dfWEO = None

for country in country_code:
    country_list += [country]*year_obs
    
    dfWEOtemp = dfWEOraw[dfWEOraw['WEO Country Code'] == country][[year for year in range(1980, end_year+1)]]
    
    if dfWEO is None:
        dfWEO = dfWEOtemp.T
        if start_year < 1980:
            nan_df = pd.DataFrame(np.nan, index=np.arange(start_year,1980), columns=dfWEO.columns)
            dfWEO = pd.concat([nan_df,dfWEO],axis=0)
    else:
        dfWEOtemp.reset_index(drop=True, inplace=True)
        if start_year < 1980:
            dfWEO = pd.concat([dfWEO,nan_df,dfWEOtemp.T], axis = 0)
        else: 
            dfWEO = pd.concat([dfWEO,dfWEOtemp.T],axis = 0)
        
subject_code = list(dfWEOraw['WEO Subject Code'][dfWEOraw['WEO Country Code']==111])
dfWEO.columns = subject_code
dfWEO.insert(0, 'YRS', year_list)
dfWEO.insert(0, 'NAM', country_list)
dfWEO.replace('--', np.nan, inplace = True)
dfWEO.replace(0, np.nan, inplace = True)
dfWEO.sort_values(by = ['NAM', 'YRS'], inplace = True)
dfWEO.reset_index(drop = True, inplace = True)

dfWEO.to_excel(f'WeoPreTSP{end_year}.xlsx', sheet_name='WeoTSP', index = False)
dfWEO

,NAM,YRS,NGDP_R,NGDP_RPCH,NGDP,NGDPD,PPPGDP,NGDP_D,NGDPRPC,NGDPRPPPPC,...,GGSB_NPGDP,GGXONLB,GGXONLB_NGDP,GGXWDN,GGXWDN_NGDP,GGXWDG,GGXWDG_NGDP,NGDP_FY,BCA,BCA_NGDPD
0,111,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,111,1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111,1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111,1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,111,1974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10579,968,2019,873.183,3.853,1063.795,250.993,607.907,121.829,44975.925,30105.501,...,-5.176,-36.445,-3.426,303.987,28.576,389.662,36.629,1063.795,-12.207,-4.863
10580,968,2020,841.072,-3.677,1066.781,251.699,593.284,126.836,43513.837,29126.824,...,-4.734,-88.015,-8.251,402.801,37.759,526.898,49.391,1066.781,-12.438,-4.941
10581,968,2021,889.088,5.709,1189.090,286.015,655.909,133.743,46302.648,30993.569,...,-7.354,-62.510,-5.257,482.167,40.549,614.411,51.671,1189.090,-20.681,-7.231
10582,968,2022,929.945,4.595,1409.784,301.273,734.353,151.599,48835.331,32688.869,...,-5.751,-53.004,-3.760,551.179,39.097,711.655,50.480,1409.784,-27.447,-9.110


In [4]:
dfCombined = dfWEO.copy()

IFS_var = {series:series for series in ['BRASS','CG','EDNA','ENDA','M0_KS','FMB','PRATE',
                                       'FIDR','IRATE','NCP','NCP_R','NEER','NFI','NGS',
                                       'NX','NM','REERAOP','TM_R','TMG_R','TX_R','TXG_R','TT',]}

IFS_df_dict = {}

for (sheet_name, var_name) in IFS_var.items():
    
    IFS_df_dict[var_name] = pd.read_excel('IMF_IFS_2023.xlsx', sheet_name = sheet_name, header = 3)
    IFS_df_dict[var_name].drop(columns =['Unnamed: 1'], index = list(range(6)), inplace = True)
    IFS_df_dict[var_name].rename(columns = {'@IFSANN':'YRS'}, inplace = True)
    IFS_df_dict[var_name] = IFS_df_dict[var_name].iloc[start_year-1969:end_year-1968,:]
    IFS_df_dict[var_name]['YRS'] = IFS_df_dict[var_name]['YRS'].astype(np.int64)

    common_code = country_code.intersection(IFS_df_dict[var_name].columns)
    dfCombined[var_name] = np.nan

    for country in common_code:
        dfCombined.loc[dfCombined['NAM'] == country, var_name] = IFS_df_dict[var_name][country].values
        
dfCombined = pd.concat([dfCombined.iloc[:,0:2],dfCombined.iloc[:,2:].astype(np.float64)],axis=1)

dfCombined

,NAM,YRS,NGDP_R,NGDP_RPCH,NGDP,NGDPD,PPPGDP,NGDP_D,NGDPRPC,NGDPRPPPPC,...,NFI,NGS,NX,NM,REERAOP,TM_R,TMG_R,TX_R,TXG_R,TT
0,111,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,227.900,192.8,59.700,55.800,121.599220,249.9758,173.7305,192.7615,119.5415,108.60818
1,111,1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,247.100,209.2,63.000,62.300,121.956089,263.3173,188.2983,196.1015,119.4213,99.16632
2,111,1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,279.700,237.3,70.800,74.200,114.002842,292.9483,213.9110,211.3870,132.4213,98.89155
3,111,1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,316.700,292.2,95.300,91.200,104.132402,306.5428,229.0090,251.1678,164.8233,140.22279
4,111,1974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,336.700,301.8,126.700,127.500,99.057700,299.5983,222.6470,271.1173,178.9290,139.52292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10579,968,2019,873.183,3.853,1063.795,250.993,607.907,121.829,44975.925,30105.501,...,244.453,NaN,427.648,471.292,97.252000,438.5224,NaN,369.3993,NaN,74.24619
10580,968,2020,841.072,-3.677,1066.781,251.699,593.284,126.836,43513.837,29126.824,...,251.016,NaN,393.392,439.327,98.615000,415.7806,NaN,334.4200,NaN,69.05534
10581,968,2021,889.088,5.709,1189.090,286.015,655.909,133.743,46302.648,30993.569,...,288.381,NaN,482.663,550.098,99.560000,477.3935,NaN,376.5631,NaN,96.81985
10582,968,2022,929.945,4.595,1409.784,301.273,734.353,151.599,48835.331,32688.869,...,346.459,NaN,606.415,700.030,103.182000,522.5683,NaN,412.9246,NaN,105.23729


In [5]:
dfWorldGDP = pd.read_excel('../WorldGDP/imf-dm-export-20240420.xlsx',header = 0,skiprows=[1],index_col=0)
seriesWorldGDP = np.full((year_obs),np.nan)
seriesWorldGDP[1980-start_year:] = dfWorldGDP.loc['World','1980':str(end_year)].values

dfCombined['WGDP_RPCH'] = np.tile(seriesWorldGDP,len(country_code))
dfCombined

,NAM,YRS,NGDP_R,NGDP_RPCH,NGDP,NGDPD,PPPGDP,NGDP_D,NGDPRPC,NGDPRPPPPC,...,NGS,NX,NM,REERAOP,TM_R,TMG_R,TX_R,TXG_R,TT,WGDP_RPCH
0,111,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,192.8,59.700,55.800,121.599220,249.9758,173.7305,192.7615,119.5415,108.60818,NaN
1,111,1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,209.2,63.000,62.300,121.956089,263.3173,188.2983,196.1015,119.4213,99.16632,NaN
2,111,1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,237.3,70.800,74.200,114.002842,292.9483,213.9110,211.3870,132.4213,98.89155,NaN
3,111,1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,292.2,95.300,91.200,104.132402,306.5428,229.0090,251.1678,164.8233,140.22279,NaN
4,111,1974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,301.8,126.700,127.500,99.057700,299.5983,222.6470,271.1173,178.9290,139.52292,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10579,968,2019,873.183,3.853,1063.795,250.993,607.907,121.829,44975.925,30105.501,...,NaN,427.648,471.292,97.252000,438.5224,NaN,369.3993,NaN,74.24619,2.8
10580,968,2020,841.072,-3.677,1066.781,251.699,593.284,126.836,43513.837,29126.824,...,NaN,393.392,439.327,98.615000,415.7806,NaN,334.4200,NaN,69.05534,-2.7
10581,968,2021,889.088,5.709,1189.090,286.015,655.909,133.743,46302.648,30993.569,...,NaN,482.663,550.098,99.560000,477.3935,NaN,376.5631,NaN,96.81985,6.5
10582,968,2022,929.945,4.595,1409.784,301.273,734.353,151.599,48835.331,32688.869,...,NaN,606.415,700.030,103.182000,522.5683,NaN,412.9246,NaN,105.23729,3.5


In [6]:
dfIncome = pd.read_excel('../List_of_countries.xlsx', sheet_name = 'Country list', header = 1)
common_code = country_code.intersection(dfIncome['code.2'])
dfCombined[['dumUpper','dumUpper-mid','dumLower-mid','dumLower']] = 0
for i in dfIncome.index:
    country = dfIncome.iloc[i]['code.2']
    if (country in common_code):
        if dfIncome.iloc[i]['Upper'] == 1:
            dfCombined.loc[(dfCombined['NAM']==country),'dumUpper'] = 1
        elif dfIncome.iloc[i]['Upper-mid'] == 1:
            dfCombined.loc[(dfCombined['NAM']==country),'dumUpper-mid'] = 1
        elif dfIncome.iloc[i]['Lower-mid'] == 1:
            dfCombined.loc[(dfCombined['NAM']==country),'dumLower-mid'] = 1
        elif dfIncome.iloc[i]['Lower'] == 1:
            dfCombined.loc[(dfCombined['NAM']==country),'dumLower'] = 1
            
dfCombined

,NAM,YRS,NGDP_R,NGDP_RPCH,NGDP,NGDPD,PPPGDP,NGDP_D,NGDPRPC,NGDPRPPPPC,...,TM_R,TMG_R,TX_R,TXG_R,TT,WGDP_RPCH,dumUpper,dumUpper-mid,dumLower-mid,dumLower
0,111,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,249.9758,173.7305,192.7615,119.5415,108.60818,NaN,1,0,0,0
1,111,1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,263.3173,188.2983,196.1015,119.4213,99.16632,NaN,1,0,0,0
2,111,1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,292.9483,213.9110,211.3870,132.4213,98.89155,NaN,1,0,0,0
3,111,1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,306.5428,229.0090,251.1678,164.8233,140.22279,NaN,1,0,0,0
4,111,1974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,299.5983,222.6470,271.1173,178.9290,139.52292,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10579,968,2019,873.183,3.853,1063.795,250.993,607.907,121.829,44975.925,30105.501,...,438.5224,NaN,369.3993,NaN,74.24619,2.8,0,1,0,0
10580,968,2020,841.072,-3.677,1066.781,251.699,593.284,126.836,43513.837,29126.824,...,415.7806,NaN,334.4200,NaN,69.05534,-2.7,0,1,0,0
10581,968,2021,889.088,5.709,1189.090,286.015,655.909,133.743,46302.648,30993.569,...,477.3935,NaN,376.5631,NaN,96.81985,6.5,0,1,0,0
10582,968,2022,929.945,4.595,1409.784,301.273,734.353,151.599,48835.331,32688.869,...,522.5683,NaN,412.9246,NaN,105.23729,3.5,0,1,0,0


In [7]:
skipcols = ['Unnamed: ' + str(i) for i in range(35,39)]
dfRegime = pd.read_excel('../Regimes/REGIMETS_2023.xlsx', sheet_name = 'Full-KS', usecols = lambda column: column not in skipcols, index_col = 1)
dfRegime.columns = dfRegime.iloc[1]
dfRegime = dfRegime.transpose()
dfRegime = dfRegime.iloc[3:,2:]
if start_year < 1970:
    nan_df = pd.DataFrame(np.nan, index=np.arange(start_year,1970), columns=dfRegime.columns)
    dfRegime = pd.concat([nan_df,dfRegime],axis=0)
else:
    dfRegime =dfRegime.loc[start_year:end_year,:]

common_code = country_code.intersection(dfRegime.columns)

dfCombined.insert(dfCombined.columns.get_loc('YRS') + 1, 'REG', np.nan)

for country in common_code:
    dfCombined.loc[dfCombined['NAM'] == country, 'REG'] = dfRegime[country].values
    

dfCombined


,NAM,YRS,REG,NGDP_R,NGDP_RPCH,NGDP,NGDPD,PPPGDP,NGDP_D,NGDPRPC,...,TM_R,TMG_R,TX_R,TXG_R,TT,WGDP_RPCH,dumUpper,dumUpper-mid,dumLower-mid,dumLower
0,111,1970,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,249.9758,173.7305,192.7615,119.5415,108.60818,NaN,1,0,0,0
1,111,1971,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,263.3173,188.2983,196.1015,119.4213,99.16632,NaN,1,0,0,0
2,111,1972,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,292.9483,213.9110,211.3870,132.4213,98.89155,NaN,1,0,0,0
3,111,1973,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,306.5428,229.0090,251.1678,164.8233,140.22279,NaN,1,0,0,0
4,111,1974,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,299.5983,222.6470,271.1173,178.9290,139.52292,NaN,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10579,968,2019,4.0,873.183,3.853,1063.795,250.993,607.907,121.829,44975.925,...,438.5224,NaN,369.3993,NaN,74.24619,2.8,0,1,0,0
10580,968,2020,4.0,841.072,-3.677,1066.781,251.699,593.284,126.836,43513.837,...,415.7806,NaN,334.4200,NaN,69.05534,-2.7,0,1,0,0
10581,968,2021,4.0,889.088,5.709,1189.090,286.015,655.909,133.743,46302.648,...,477.3935,NaN,376.5631,NaN,96.81985,6.5,0,1,0,0
10582,968,2022,4.0,929.945,4.595,1409.784,301.273,734.353,151.599,48835.331,...,522.5683,NaN,412.9246,NaN,105.23729,3.5,0,1,0,0


In [8]:
dfSOVD = pd.read_excel('../SovereignDebt/SOVD data from Schuler.xlsx', sheet_name = 'SOVD')

common_code = country_code.intersection(dfSOVD['NAM'])

dfCombined['SOVD'] = np.nan

for i in dfSOVD.index:
    country = dfSOVD.iloc[i]['NAM']
    year = dfSOVD.iloc[i]['YRS']
    if (country in common_code) and \
        (year in list(range(start_year,end_year+1))):
        
        dfCombined.loc[(dfCombined['NAM']==country)&(dfCombined['YRS']==year),'SOVD'] = 1
        
dfCombined.loc[(dfCombined['NAM'].isin(common_code))&(dfCombined['SOVD'].isna()),'SOVD'] = 0
        
dfCombined

FileNotFoundError: [Errno 2] No such file or directory: '../SovereignDebt/SOVD data from Schuler.xlsx'

In [ ]:
dfCBGOV = pd.read_excel('../Cbturn/CBGOV-data-from-Dreher.xlsx', sheet_name = 'CBGOV')


common_code = country_code.intersection(dfCBGOV['NAM'])

dfCombined['CBGOV'] = np.nan

for i in dfCBGOV.index:
    country = dfCBGOV.iloc[i]['NAM']
    year = dfCBGOV.iloc[i]['YRS']
    if (country in common_code) and\
        (year in list(range(start_year,end_year+1))) and\
        (isinstance(dfCBGOV.iloc[i]['CBGOV'],int)):
        
        dfCombined.loc[(dfCombined['NAM']==country)&(dfCombined['YRS']==year),'CBGOV'] = dfCBGOV.iloc[i]['CBGOV']
        
dfCombined.loc[dfCombined['CBGOV']<0,'CBGOV'] = 0
dfCombined

In [ ]:
avg5y_series = {'CBGOV':'CBTURN5'}


for (orig_series, avg_series) in avg5y_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,avg_series,dfCombined[orig_series].shift(1).rolling(5,min_periods=1).sum()/5)
    dfCombined[avg_series] = dfCombined[avg_series].fillna(0)
    dfCombined.loc[dfCombined['YRS'] == start_year, avg_series] = 0
    dfCombined.loc[dfCombined['YRS'] == start_year+1, avg_series] = 0#dfCombined.loc[dfCombined['YRS'] == start_year, orig_series].values/5
    dfCombined.loc[dfCombined['YRS'] == start_year+2, avg_series] = 0#dfCombined.loc[dfCombined['YRS'] == start_year+1, avg_series].values+dfCombined.loc[dfCombined['YRS'] == start_year+1, orig_series].values/5
    dfCombined.loc[dfCombined['YRS'] == start_year+3, avg_series] = 0#dfCombined.loc[dfCombined['YRS'] == start_year+2, avg_series].values+dfCombined.loc[dfCombined['YRS'] == start_year+2, orig_series].values/5
    dfCombined.loc[dfCombined['YRS'] == start_year+4, avg_series] = 0#dfCombined.loc[dfCombined['YRS'] == start_year+3, avg_series].values+dfCombined.loc[dfCombined['YRS'] == start_year+3, orig_series].values/5

In [ ]:
dfCURCON = pd.read_excel('../CURCON-KAPCON/CURCON-data-from-AREAR.xlsx', sheet_name = 'CURCON')

common_code = country_code.intersection(dfCURCON['NAM'])

dfCombined[['CURCON','CURCON1','CURCON2','CURCON3']] = np.nan

for i in dfCURCON.index:
    country = dfCURCON.iloc[i]['NAM']
    year = dfCURCON.iloc[i]['YRS']
    if (country in common_code) and \
        (year in list(range(start_year,end_year+1))):
        
        dfCombined.loc[(dfCombined['NAM']==country)&(dfCombined['YRS']==year),'CURCON'] = dfCURCON.iloc[i]['CURCON']
        dfCombined.loc[(dfCombined['NAM']==country)&(dfCombined['YRS']==year),'CURCON1'] = dfCURCON.iloc[i]['CURCON1']
        dfCombined.loc[(dfCombined['NAM']==country)&(dfCombined['YRS']==year),'CURCON2'] = dfCURCON.iloc[i]['CURCON2']
        dfCombined.loc[(dfCombined['NAM']==country)&(dfCombined['YRS']==year),'CURCON3'] = dfCURCON.iloc[i]['CURCON3']
        
dfCombined

In [ ]:
dfKAPCON = pd.read_excel('../CURCON-KAPCON/KAPCON-data-from-Chinn-Ito.xlsx', sheet_name = 'KAPCON')

common_code = country_code.intersection(dfKAPCON['NAM'])

dfCombined['KAPCON'] = np.nan

for i in dfKAPCON.index:
    country = dfKAPCON.iloc[i]['NAM']
    year = dfKAPCON.iloc[i]['YRS']
    if (country in common_code) and \
        (year in list(range(start_year,end_year+1))) and \
        (isinstance(dfKAPCON.iloc[i]['KAPCON'],int)):
        dfCombined.loc[(dfCombined['NAM']==country)&(dfCombined['YRS']==year),'KAPCON'] = 1-dfKAPCON.iloc[i]['KAPCON']

dfCombined

In [ ]:
dfTrade = pd.read_excel('../Trade/Trade-data-from-IMF-DOTS-database-2023Q4.xlsx', sheet_name = 'DOT_2023Q4_03-12-2024 23-29-45-', index_col = 1)
dfTradeImportsCIF = dfTrade.loc[dfTrade['Indicator Code']=='TMG_CIF_USD',:].copy().transpose()
dfTradeImportsFOB = dfTrade.loc[dfTrade['Indicator Code']=='TMG_FOB_USD',:].copy().transpose()
dfTradeExportsCIF = dfTrade.loc[dfTrade['Indicator Code']=='TXG_FOB_USD',:].copy().transpose()
dfTradeExportsFOB = dfTrade.loc[dfTrade['Indicator Code']=='TXG_FOB_USD',:].copy().transpose()

dfTradeImportsCIF.drop(index = ['Country Name','Indicator Name','Indicator Code','Counterpart Country Name','Attribute'], inplace = True)
dfTradeImportsFOB.drop(index = ['Country Name','Indicator Name','Indicator Code','Counterpart Country Name','Attribute'], inplace = True)
dfTradeExportsCIF.drop(index = ['Country Name','Indicator Name','Indicator Code','Counterpart Country Name','Attribute'], inplace = True)
dfTradeExportsFOB.drop(index = ['Country Name','Indicator Name','Indicator Code','Counterpart Country Name','Attribute'], inplace = True)

In [ ]:
dfBNKSTRT = pd.read_excel('../Crisis/BNKSTRT-data-from-Metrick-Schmelzing-plus-Schuler.xlsx', sheet_name = 'BNKSTRT')

common_code = country_code.intersection(dfBNKSTRT['NAM'])

dfCombined['BNKSTRT'] = np.nan

for i in dfBNKSTRT.index:
    country = dfBNKSTRT.iloc[i]['NAM']
    year = dfBNKSTRT.iloc[i]['YRS']
    if (country in common_code) and \
        (year in list(range(start_year,end_year+1))):
        
        dfCombined.loc[(dfCombined['NAM']==country)&(dfCombined['YRS']==year),'BNKSTRT'] = 1
        
dfCombined.loc[(dfCombined['NAM'].isin(common_code))&(dfCombined['BNKSTRT'].isna()),'BNKSTRT'] = 0
        
dfCombined

In [ ]:
per_capita_series = {}

for (orig_series, capita_series) in per_capita_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,capita_series,dfCombined[orig_series]/dfCombined['LP'])

In [ ]:
natural_log_series = {series:series+'_LN' for series in ['NGDP_R']}

for (orig_series, ln_series) in natural_log_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,ln_series, np.log(dfCombined[orig_series]))

In [ ]:
# Percentage of NGDP

per_NGDP_series = {series:series+'_NGDP' for series in ['NX','NM','NGS','NCP','NFI']}

for (orig_series, NGDP_series) in per_NGDP_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,NGDP_series,dfCombined[orig_series]/dfCombined['NGDP']*100)


In [ ]:
# Ratio of NGDP

per_NGDP_series = {series:series+'_NGDP' for series in ['FMB','GGSB']}

for (orig_series, NGDP_series) in per_NGDP_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,NGDP_series,dfCombined[orig_series]/dfCombined['NGDP'])

In [ ]:
real_series = {series:series+'_R' for series in ['FMB','NX','NM']}

for (orig_series, real_series) in real_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,real_series,dfCombined[orig_series]/dfCombined['PCPI'])

In [ ]:
dfCombined.insert(dfCombined.columns.get_loc('NX_NGDP') + 1,'OPEN',dfCombined['NM_NGDP'] + dfCombined['NM_NGDP'])

In [ ]:
for country in set(dfCombined['NAM']):
    numerator = dfCombined.loc[(dfCombined['NAM'] == country) & (dfCombined['YRS'] == start_year), 'PPPPC'].values
    denominator = dfCombined.loc[(dfCombined['NAM'] == 111) & (dfCombined['YRS'] == start_year), 'PPPPC'].values 
    dfCombined.loc[(dfCombined['NAM'] == country), 'GAP_US'] = np.divide(numerator, denominator)[0]*100
   

In [ ]:
growth_pct_series = {series:series+'PCH' for series in ['FMB','FMB_R','FMB_NGDP','LP','NGDPRPC','TT','NX_R','NM_R','REERAOP']}

for (orig_series, growth_series) in growth_pct_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,growth_series, dfCombined[orig_series].pct_change()*100)
    dfCombined.loc[dfCombined['YRS'] == start_year, growth_series] = np.nan
    dfCombined.loc[dfCombined[orig_series].isna(), growth_series] = np.nan
    dfCombined.loc[dfCombined[orig_series].shift(1).isna(), growth_series] = np.nan

In [ ]:
# Ex-Post Real Interest Rate
dfCombined.insert(dfCombined.columns.get_loc('IRATE') + 1,'RRATE', dfCombined['IRATE']-dfCombined['PCPIPCH'].shift(-1))
dfCombined.loc[dfCombined['YRS'] == end_year, 'RRATE'] = np.nan

In [ ]:
# Rescaled series
rescale_series = {series:series+'_X' for series in ['PCPIPCH','FMBPCH']}

for (orig_series, rescaled_series) in rescale_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,rescaled_series, 100*dfCombined[orig_series]/(100+dfCombined[orig_series]))

In [ ]:
avg3y_series = {series:series+'_M3' for series in ['GGSB_NGDP','GGSB_NPGDP','NFI_NGDP']}

for (orig_series, avg_series) in avg3y_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,avg_series,dfCombined[orig_series].shift(1).rolling(3,min_periods=1).mean())
    dfCombined.loc[dfCombined['YRS'] == start_year, avg_series] = np.nan
    dfCombined.loc[dfCombined['YRS'] == start_year+1, avg_series] = np.nan
    dfCombined.loc[dfCombined['YRS'] == start_year+2, avg_series] = np.nan

In [ ]:
hp_filter_series = {series:series+'_HP' for series in ['NGDP_R','NGDP_R_LN']}

for (orig_series, hp_series) in hp_filter_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,hp_series, np.nan)
    for country in set(dfCombined['NAM']):
        [cycle, trend] = hpfilter.hpfilter(dfCombined.loc[dfCombined['NAM'] == country,orig_series], lamb = 100)
        dfCombined.loc[dfCombined['NAM'] == country,hp_series] = trend

hp_gap_series = {series:series+'_GAP' for series in ['NGDP_R_LN']}
for (orig_series, gap_series) in hp_gap_series.items():
    dfCombined.insert(dfCombined.columns.get_loc(orig_series) + 1,gap_series,dfCombined[orig_series+'_HP'] - dfCombined[orig_series])


In [ ]:
dfCombined.loc[:,['dumDOLLAR','dumCURRENCYBOARD','dumPEGGED','dumINTERMEDIATE','dumFLEXIBLE']] = 0
dfCombined.loc[dfCombined['REG'] == 1, 'dumDOLLAR'] = 1
dfCombined.loc[dfCombined['REG'] == 2, 'dumCURRENCYBOARD'] = 1
dfCombined.loc[dfCombined['REG'] == 3, 'dumPEGGED'] = 1
dfCombined.loc[dfCombined['REG'] == 4, 'dumINTERMEDIATE'] = 1
dfCombined.loc[(dfCombined['REG'] == 5) | (dfCombined['REG'] == 6), 'dumFLEXIBLE'] = 1
dfCombined

In [ ]:
dfCombined.to_excel('WEO-base.xlsx')